In [32]:
import pickle
from transformers import BertJapaneseTokenizer, BertForMaskedLM
import pandas as pd
import json
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import collections
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import DistilBertConfig

In [31]:
max_seq_length = 28

In [33]:
model = TFDistilBertForSequenceClassification.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', num_labels=2)

You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some layers from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing TFDistilBertForSequenceClassification: ['mlm___cls', 'bert', 'nsp___cls']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-wor

In [34]:
class MyModel(tf.keras.Model):
    def __init__(self,model):
        super().__init__()
        #self.model = model.distilbert(input_ids,input_mask)[0]
        self.model = model
        self.model.trainable = False
        self.layersG = tf.keras.layers.GlobalMaxPool1D()
        self.layers1 = tf.keras.layers.Dense(100, activation="relu")
        self.layersD2 = tf.keras.layers.Dropout(0.2)
        self.layers3 = tf.keras.layers.Dense(200, activation="relu")
        self.layersD4 = tf.keras.layers.Dropout(0.2)
        self.layers5 = tf.keras.layers.Dense(100, activation="relu")
        self.layersD6 = tf.keras.layers.Dropout(0.2)
        self.layers7 = tf.keras.layers.Dense(50, activation="relu")
        self.layersD8 = tf.keras.layers.Dropout(0.2)
        self.layers9 = tf.keras.layers.Dense(2, activation="softmax")
        
        self.Bidirectional=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))
    def call(self, inputs):
        input_ids1 = inputs["input_ids1"]
        input_mask1 = inputs["input_mask1"]
        embedding_layer1  = self.model.distilbert(input_ids1,input_mask1)[0]
        input_ids2 = inputs["input_ids2"]
        input_mask2 = inputs["input_mask2"]        
        embedding_layer2  = self.model.distilbert(input_ids2,input_mask2)[0]
        embedding_layer = tf.concat([embedding_layer1,embedding_layer2],axis = 1)
        X =self.Bidirectional(embedding_layer)
        X = self.layersG(X)
        X = self.layers1(X)
        X = self.layersD2(X)
        X = self.layers3(X)
        X = self.layersD4(X)
        X = self.layers5(X)
        X = self.layersD6(X)
        X = self.layers7(X)
        X = self.layersD8(X)
        X = self.layers9(X)        
        return X

In [35]:
train_output_file = "../input/text-data-similarity/texts_train_data.tfrecord"
test_output_file = "../input/text-data-similarity/texts_test_data.tfrecord"

In [36]:
def select_data_and_label_from_record(record):
    x = {
        "input_ids1": record["input_ids1"],
        "input_mask1": record["input_mask1"],
        "input_ids2": record["input_ids2"],
        "input_mask2": record["input_mask2"],
        #"segment_ids": record["segment_ids"],
    }
    y = record["label_ids"]
    return (x, y)
def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    return tf.io.parse_single_example(record, name_to_features)

In [37]:
def create_train_test_data(file_name,isTrain = False):
    dataset = tf.data.TFRecordDataset(file_name)
    if isTrain :
        dataset = dataset.repeat(500)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    name_to_features = {
            "input_ids1": tf.io.FixedLenFeature([max_seq_length], tf.int64),
            "input_mask1": tf.io.FixedLenFeature([max_seq_length], tf.int64),
            "input_ids2": tf.io.FixedLenFeature([max_seq_length], tf.int64),
            "input_mask2": tf.io.FixedLenFeature([max_seq_length], tf.int64),
            #"segment_ids": tf.io.FixedLenFeature([max_seq_length], tf.int64),
            "label_ids": tf.io.FixedLenFeature([], tf.int64),
        }
    drop_remainder=False
    dataset = dataset.apply(
        tf.data.experimental.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=1000,
            drop_remainder=drop_remainder,
            num_parallel_calls=tf.data.experimental.AUTOTUNE,
        )
    )
    dataset.cache()
    re_dataset = dataset.map(select_data_and_label_from_record)
    return re_dataset

In [38]:
train_dataset = create_train_test_data(train_output_file,True)
test_dataset = create_train_test_data(test_output_file)

In [39]:
test_dataset

<MapDataset shapes: ({input_ids1: (None, 28), input_mask1: (None, 28), input_ids2: (None, 28), input_mask2: (None, 28)}, (None,)), types: ({input_ids1: tf.int64, input_mask1: tf.int64, input_ids2: tf.int64, input_mask2: tf.int64}, tf.int64)>

In [40]:
mode2 = MyModel(model)

In [ ]:
learning_rate = 1e-3
epsilon = 1e-6
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metric = tf.keras.metrics.SparseCategoricalAccuracy("accuracy")
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=epsilon)
mode2.compile(optimizer=optimizer, loss=loss, metrics=[metric])
history = mode2.fit(
    train_dataset,
    steps_per_epoch = 20,
    validation_data=test_dataset,
    validation_steps=4,
    #batch_size=50,
    #shuffle=True,
    epochs=200)

Epoch 1/200
20/20 [==============================] - 144s 3s/step - loss: 0.6999 - accuracy: 0.4976 - val_loss: 0.6930 - val_accuracy: 0.5033
Epoch 2/200
20/20 [==============================] - 52s 3s/step - loss: 0.6939 - accuracy: 0.4994 - val_loss: 0.6931 - val_accuracy: 0.5063
Epoch 3/200
20/20 [==============================] - 52s 3s/step - loss: 0.6935 - accuracy: 0.5027 - val_loss: 0.6928 - val_accuracy: 0.5073
Epoch 4/200
20/20 [==============================] - 53s 3s/step - loss: 0.6935 - accuracy: 0.4998 - val_loss: 0.6930 - val_accuracy: 0.5017
Epoch 5/200
20/20 [==============================] - 52s 3s/step - loss: 0.6933 - accuracy: 0.5031 - val_loss: 0.6929 - val_accuracy: 0.5120
Epoch 6/200
20/20 [==============================] - 53s 3s/step - loss: 0.6930 - accuracy: 0.5127 - val_loss: 0.6925 - val_accuracy: 0.5272
Epoch 7/200
20/20 [==============================] - 53s 3s/step - loss: 0.6928 - accuracy: 0.5045 - val_loss: 0.6910 - val_accuracy: 0.5335
Epoch 8/200
